In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
from IPython.display import display

In [4]:
import os
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
from datascifuncs.tidbit_tools import load_json, write_json, print_json, check_directory_name

In [6]:
main_dir = 'EmotionFaceClassifier'
check_directory_name(main_dir)

Directory set to /Users/dsl/Documents/GitHub/EmotionFaceClassifier, matches target dir string EmotionFaceClassifier.


True

In [7]:
from utils.feature_extraction import perform_unsupervised_analysis

In [8]:
common_dicts = load_json('./configs/input_mappings.json')

In [9]:
emotion_colors = common_dicts['plotly_styles']['Training']['color']

In [10]:
# Read in FER 2013 data
fer2013_path = 'data/fer2013_paths.csv'
fer2013 = pd.read_csv(fer2013_path)

In [11]:
fer2013.head()

,emotion_id,pixels,Usage,emotion,image,usage,emo_count_id,img_path,color
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training,Angry,[[ 70 80 82 ... 52 43 41]\n [ 65 61 58 ...,Training,1,data/Training/Angry/Angry-1.jpg,red
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training,Angry,[[151 150 147 ... 129 140 120]\n [151 149 149 ...,Training,2,data/Training/Angry/Angry-2.jpg,red
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training,Fear,[[231 212 156 ... 44 27 16]\n [229 175 148 ...,Training,1,data/Training/Fear/Fear-1.jpg,slategray
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training,Sad,[[ 24 32 36 ... 173 172 173]\n [ 25 34 29 ...,Training,1,data/Training/Sad/Sad-1.jpg,blue
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training,Neutral,[[ 4 0 0 ... 27 24 25]\n [ 1 0 0 ... 26 23...,Training,1,data/Training/Neutral/Neutral-1.jpg,sienna


In [12]:
# Select training data
print(fer2013.shape)
train_df = fer2013[fer2013['usage']=='Training']
print(train_df.shape)

(35887, 9)
(28709, 9)


In [13]:
train_df['emotion'].unique()

array(['Angry', 'Fear', 'Sad', 'Neutral', 'Happy', 'Surprise', 'Disgust'],
      dtype=object)

In [14]:
from datascifuncs import load_json

In [15]:
test_dict = load_json('configs/unsupervised_models_test.json')

In [16]:
test_dict

{'FastICA': {'module': 'sklearn.decomposition',
  'class': 'FastICA',
  'normalization': 'standard',
  'total_components': 30,
  'components_for_reconstruction': [1, 10, 30],
  'params': {'algorithm': 'parallel',
   'fun': 'logcosh',
   'max_iter': 500,
   'tol': 0.0001,
   'random_state': 42}},
 'NMF': {'module': 'sklearn.decomposition',
  'class': 'NMF',
  'normalization': 'none',
  'total_components': 30,
  'components_for_reconstruction': [1, 10, 30],
  'params': {'init': 'random',
   'solver': 'mu',
   'beta_loss': 'frobenius',
   'max_iter': 1000,
   'tol': 1e-05,
   'random_state': 42}},
 'PCA': {'module': 'sklearn.decomposition',
  'class': 'PCA',
  'normalization': 'standard',
  'total_components': 30,
  'components_for_reconstruction': [1, 10, 30],
  'params': {'svd_solver': 'randomized', 'whiten': True, 'random_state': 42}}}

In [17]:
test_dict['PCA']

{'module': 'sklearn.decomposition',
 'class': 'PCA',
 'normalization': 'standard',
 'total_components': 30,
 'components_for_reconstruction': [1, 10, 30],
 'params': {'svd_solver': 'randomized', 'whiten': True, 'random_state': 42}}

In [18]:
from utils.decomposition_feature_extract import create_X_y

In [19]:
X, y = create_X_y(train_df, 'img_path', 'emotion')

In [20]:
from utils.decomposition_feature_extract import run_single_analysis

In [21]:
run_single_analysis(X, y, test_dict['PCA'])

Analysis type: PCA
Normalization: standard
Total components: 30
Parameters: {'svd_solver': 'randomized', 'whiten': True, 'random_state': 42}
Analysis settings saved to models/unsupervised/pca_standard_30/pca_standard_30_info.json
Averaged reconstructions saved to models/unsupervised/pca_standard_30/pca_avg_reconstructions.npz
Metrics saved to models/unsupervised/pca_standard_30/pca_standard_30_metrics.csv


{'Overall': [array([-1.12383543e-16,  9.57315510e-17,  2.10373515e-18, ...,
          3.63319702e-17,  1.26382663e-16,  7.87663192e-17]),
  array([-9.93589473e-17,  6.55019596e-17, -9.12649810e-18, ...,
          5.09595037e-17,  1.60916403e-16,  7.51621258e-17]),
  array([-9.95252352e-17,  6.42683355e-17, -2.49818550e-18, ...,
          5.60119485e-17,  1.74331582e-16,  1.20763679e-16])],
 'Angry': [array([-0.05150946, -0.04420727, -0.03095065, ..., -0.0705843 ,
         -0.06814493, -0.07050149]),
  array([-0.05150946, -0.04420727, -0.03095065, ..., -0.0705843 ,
         -0.06814493, -0.07050149]),
  array([-0.05150946, -0.04420727, -0.03095065, ..., -0.0705843 ,
         -0.06814493, -0.07050149])],
 'Disgust': [array([-0.14061367, -0.15339212, -0.18702143, ..., -0.05945859,
         -0.07754646, -0.08082209]),
  array([-0.14061367, -0.15339212, -0.18702143, ..., -0.05945859,
         -0.07754646, -0.08082209]),
  array([-0.14061367, -0.15339212, -0.18702143, ..., -0.05945859,
     